In [1]:
def find_sc(square, cyl_side, d_opt):
    sigma_ind = 0
    if square == 2*d_opt or square == 4*d_opt:
        if cyl_side[-1][1] == square:
            return cyl_side[-1][0]
        else:
            return cyl_side[0][0]
    if square > cyl_side[-1][1]:
        return cyl_side[0][0]
    while cyl_side[sigma_ind][1] < square:
        sigma_ind += 1
    return cyl_side[sigma_ind][0]

def dist_on_sc(square, cyl_side, d_opt):
    width = 2*d_opt
    sigma_ind = 0
    if square == 2*d_opt or square == 4*d_opt:
        if cyl_side[-1][1] == square:
            return (square - cyl_side[-2][1])%width
        else:
            return (square - cyl_side[-1][1])%width
    if square > cyl_side[-1][1]:
        return (square - cyl_side[-1][1])%width
    while cyl_side[sigma_ind][1] < square:
        sigma_ind += 1
    return (square - cyl_side[sigma_ind-1][1])%width


def get_square(sc, dist_on_sc, cyl_side, d_opt):
    sc_index = 0
    while sc != cyl_side[sc_index][0]:
        sc_index +=1
    return cyl_side[sc_index-1][1] + dist_on_sc


def vert_perm(vert_perm_init, surf_coord, d_opt, print_results = False, fast_check_break = False):
    '''
    It's going to be much more convenient in terms of python to label the squares from 0.
    In other words the first cylinder is (1, ..., 2d_opt) and 
    the second cylinder is (2d_opt + 1, ..., 4d_opt)
    This function is written in a way that requires us to always start in a square contained in C_1.
    Given the topology of the surface, this is never a problem.
    '''
    width = 2*d_opt
    t_partition = surf_coord[0][0]
    len_t_part = len(t_partition)
    t_cyl_diag = surf_coord[0][1]
    t_start = surf_coord[1]
    s_partition = surf_coord[2][0]
    len_s_part = len(s_partition)
    s_cyl_diag = surf_coord[2][1]
    #t_i_sc_length gives the coordinates of the right endpoint of the saddle connection
    #This is for the bottom of C_1
    C_1_bot_coord = tuple([(t_cyl_diag[0][i], sum(t_partition[:i+1])) for i in range(len_t_part)])
    if print_results:
        print(C_1_bot_coord)
    #t_i_sc_length gives the coordinates of the right endpoint of the saddle connection - width
    #This is for the bottom of C_2
    C_2_bot_coord_pre = tuple([(s_cyl_diag[0][i], sum(s_partition[:i+1])) for i in range(len_s_part)])
    if print_results:
        print(C_2_bot_coord_pre)
    C_2_bot_coord = tuple([(tau[0], tau[1]+width) for tau in C_2_bot_coord_pre])
    if print_results:
        print(C_2_bot_coord)
    #The top of C_1 is given in the reverse order as usual
    C_1_top_pre = s_cyl_diag[1][::-1]
    if print_results:
        print(C_1_top_pre)
    C_1_top_start_index = C_1_top_pre.index(s_cyl_diag[0][0])
    if print_results:
        print(C_1_top_start_index)
    C_1_top = [C_1_top_pre[(i+C_1_top_start_index)%len_s_part] for i in range(len_s_part)]
    if print_results:
        print(C_1_top)
    s_part_rearrange = tuple([s_partition[s_cyl_diag[0].index(sigma)] for sigma in C_1_top])
    if print_results:
        print(s_part_rearrange)
    #This are the coordinates of the right endpoints of the saddle connections on the top of C_1
    C_1_top_coord = tuple([(C_1_top[i], sum(s_part_rearrange[:i+1])) for i in range(len_s_part)])
    if print_results:
        print(C_1_top_coord)
    C_2_top_pre = t_cyl_diag[1][::-1]
    if print_results:
        print(C_2_top_pre)
    C_2_top_start_index = C_2_top_pre.index(t_cyl_diag[0][0])
    if print_results:
        print(C_2_top_start_index)
    C_2_top = [C_2_top_pre[(i+C_2_top_start_index)%len_t_part] for i in range(len_t_part)]
    if print_results:
        print(C_2_top)
    t_part_rearrange = tuple([t_partition[t_cyl_diag[0].index(tau)] for tau in C_2_top])
    if print_results:
        print(t_part_rearrange)
    #These are the coordinates of the right endpoints of the saddle connections on the top of C_1
#    The following line was a bug that is now fixed
#    C_2_top_coord_pre = tuple([(C_2_top[i], 1 + (sum(t_part_rearrange[:i+1]) + 2*s_partition[0] + t_start)%width) 
#                           for i in range(len_t_part)])
    C_2_top_coord_pre = tuple([(C_2_top[i], (sum(t_part_rearrange[:i+1]) + 2*s_partition[0] + t_start)%width) 
                           for i in range(len_t_part)])
    if print_results:
        print(C_2_top_coord_pre)
    C_2_top_coord_unsort = [(tau[0], tau[1]+width) for tau in C_2_top_coord_pre]
    if print_results:
        print(C_2_top_coord_unsort)
    C_2_top_coord = tuple(sorted(C_2_top_coord_unsort, key=lambda x: x[1]))
    if print_results:
        print(C_2_top_coord)
        print('\n')
    if print_results:
        print(C_2_top_coord)
        print(C_2_bot_coord)
        print(C_1_top_coord)
        print(C_1_bot_coord)
    vert_perm = [vert_perm_init]
    C_1_next_square = 0
    while vert_perm_init != C_1_next_square:
    #    print(find_sc(vert_perm[-1], C_1_top_coord))
    #    print(dist_on_sc(vert_perm[-1], C_1_top_coord))
        if len(vert_perm) > 4*d_opt+3:
            return 'Abort: Serious error in code'
        if fast_check_break and len(vert_perm) > width:
            return False
        C_2_next_square_pre = get_square(find_sc(vert_perm[-1], C_1_top_coord, d_opt), 
                                         dist_on_sc(vert_perm[-1], C_1_top_coord, d_opt), 
                                         C_2_bot_coord, d_opt)
        C_2_next_square = (C_2_next_square_pre)%width + width
        if C_2_next_square == width:
            C_2_next_square = 2*width
#        print(C_2_next_square)
        vert_perm += [C_2_next_square]
        C_1_next_square_pre = get_square(find_sc(vert_perm[-1], C_2_top_coord, d_opt), 
                                         dist_on_sc(vert_perm[-1], C_2_top_coord, d_opt), 
                                         C_1_bot_coord, d_opt)
#    print(C_1_next_square_pre)
        C_1_next_square = (C_1_next_square_pre)%width# + width
        if C_1_next_square == 0:
            C_1_next_square = width
        vert_perm += [C_1_next_square]
#        print(vert_perm)
#        print(C_1_next_square)
        found = 1
    return vert_perm[:-1]


def vert_perm_check(surf_coord, d_opt):
    vert_perm_answer = vert_perm(1, surf_coord, d_opt, print_results = False, fast_check_break = True)
    if vert_perm_answer == False:
        return False
    elif len(vert_perm_answer) == 2*d_opt:
        return True
    return False

def vert_perm_check_file(d_opt, root_dir):
    admissible_list_filename = root_dir + 'admissible_list'
    with open(admissible_list_filename, 'r') as file:
        admissible_list = eval(file.read())
    print('admissible_list read with ' + str(len(admissible_list)) + ' candidates')
    admissible_list_vert_perm_check = []
    for config in admissible_list:
        if vert_perm_check(config, d_opt):
            admissible_list_vert_perm_check += [config]
    filename = root_dir + 'admissible_list_vert_perm_check'
    with open(filename, 'w') as file:
        file.write(str(admissible_list_vert_perm_check))
    print('admissible_list_vert_perm_check written with ' + str(len(admissible_list_vert_perm_check)) + ' elements')
    return len(admissible_list_vert_perm_check)

In [ ]:
def missing_square_search(vert_perm, d_opt):
    #This only works when we have a 2*d_opt cycle.  In general, this fails.
    for sq in range(1,2*d_opt+1):
        if sq not in vert_perm:
            return sq

def all_vert_perm_check(surf_coord, d_opt, trigger_check = True):
    vert_perm1 = vert_perm(1, surf_coord, d_opt)
    vert_perm2 = vert_perm(missing_square_search(vert_perm1, d_opt), surf_coord, d_opt)
    if len([i for i in range(4*d_opt+1) if i in vert_perm1 and i in vert_perm2]) > 0:
        print('ERROR: ', (vert_perm1, vert_perm2))
    if trigger_check:
        if len(vert_perm2) < 2*d_opt:
            return False
        else:
            return True
    return (vert_perm1, vert_perm2)

def all_vert_perm_check_file(d_opt, root_dir):
    admissible_list_vert_perm_check_filename = root_dir + 'admissible_list_vert_perm_check'
    with open(admissible_list_vert_perm_check_filename, 'r') as file:
        admissible_list_vert_perm_check = eval(file.read())
    print('admissible_list_vert_perm_check read with ' + str(len(admissible_list_vert_perm_check)) + ' candidates')
    admissible_list_all_vert_perm_check = []
    for config in admissible_list_vert_perm_check:
        if all_vert_perm_check(config, d_opt, trigger_check = True):
            admissible_list_all_vert_perm_check += [config]
    filename = root_dir + 'admissible_list_all_vert_perm_check'
    with open(filename, 'w') as file:
        file.write(str(admissible_list_all_vert_perm_check))
    print('admissible_list_all_vert_perm_check written with ' 
          + str(len(admissible_list_all_vert_perm_check)) + ' elements')
    return len(admissible_list_vert_perm_check)